In [10]:
import OpenSSL
from OpenSSL import crypto, SSL

# Create a private key for the root CA
root_pkey = crypto.PKey()
root_pkey.generate_key(crypto.TYPE_RSA, 2048)

# Create a self-signed certificate for the root CA
root_cert = crypto.X509()
root_cert.set_version(3)
root_cert.set_serial_number(1)
root_cert.get_subject().CN = "My Root CA"
root_cert.gmtime_adj_notBefore(0)
root_cert.gmtime_adj_notAfter(10*365*24*60*60) # 10 years
root_cert.set_issuer(root_cert.get_subject())
root_cert.set_pubkey(root_pkey)
root_cert.add_extensions([
    crypto.X509Extension(b"basicConstraints", True, b"CA:TRUE, pathlen:0"),
    crypto.X509Extension(b"keyUsage", True, b"keyCertSign, cRLSign"),
    crypto.X509Extension(b"subjectKeyIdentifier", False, b"hash", subject=root_cert),
])
root_cert.sign(root_pkey, "sha256")

# Create a private key for the intermediate CA
int_pkey = crypto.PKey()
int_pkey.generate_key(crypto.TYPE_RSA, 2048)

# Create a certificate signing request (CSR) for the intermediate CA
int_req = crypto.X509Req()
int_req.get_subject().CN = "My Intermediate CA"
int_req.set_pubkey(int_pkey)
int_req.sign(int_pkey, "sha256")

# Create a signed certificate for the intermediate CA, using the root CA's private key
int_cert = crypto.X509()
int_cert.set_version(3)
int_cert.set_serial_number(2)
int_cert.gmtime_adj_notBefore(0)
int_cert.gmtime_adj_notAfter(10*365*24*60*60) # 10 years
int_cert.set_issuer(root_cert.get_subject())
int_cert.set_subject(int_req.get_subject())
int_cert.set_pubkey(int_req.get_pubkey())
int_cert.add_extensions([
    crypto.X509Extension(b"basicConstraints", True, b"CA:TRUE, pathlen:1"),
    crypto.X509Extension(b"keyUsage", True, b"keyCertSign, cRLSign"),
    crypto.X509Extension(b"authorityKeyIdentifier", False, b"keyid:always,issuer:always", issuer=root_cert),
])
int_cert.sign(root_pkey, "sha256")

# Write the root CA and intermediate CA certificates to files
with open("root.crt", "wb") as f:
    f.write(crypto.dump_certificate(crypto.FILETYPE_PEM, root_cert))

with open("int.crt", "wb") as f:
    f.write(crypto.dump_certificate(crypto.FILETYPE_PEM, int_cert))

# Issue a certificate for a client
client_pkey = crypto.PKey()
client_pkey.generate_key(crypto.TYPE_RSA, 2048)

client_req = crypto.X509Req()
client_req.get_subject().CN = "www.example.com"
client_req.set_pubkey(client_pkey)
client_req.sign(client_pkey, "sha256")

client_cert = crypto.X509()
client_cert.set_version(3)
client_cert.set_serial_number(3)
client_cert.gmtime_adj_notBefore(0)
client_cert.gmtime_adj_notAfter(365*24*60*60) # 1 year
client_cert.set_issuer(int_cert.get_subject())
client_cert.set_subject(client_req.get_subject())
client_cert.set_pubkey(client_req.get_pubkey())
client_cert.add_extensions([
    crypto.X509Extension(b"basicConstraints", False, b"CA:FALSE"),
crypto.X509Extension(b"keyUsage", True, b"digitalSignature, keyEncipherment"),
crypto.X509Extension(b"extendedKeyUsage", True, b"serverAuth, clientAuth"),
crypto.X509Extension(b"subjectAltName", False, b"DNS:www.example.com"),
])
client_cert.sign(int_pkey, "sha256")

with open("client.crt", "wb") as f:
    f.write(crypto.dump_certificate(crypto.FILETYPE_PEM, client_cert))

In [12]:
client_cert.

### Verify the client's certificate

In [11]:
import OpenSSL
from OpenSSL import crypto, SSL

# Load the root CA certificate
with open("root.crt", "rb") as f:
    root_cert = crypto.load_certificate(crypto.FILETYPE_PEM, f.read())

# Load the intermediate CA certificate
with open("int.crt", "rb") as f:
    int_cert = crypto.load_certificate(crypto.FILETYPE_PEM, f.read())

# Load the client certificate to be verified
with open("client.crt", "rb") as f:
    client_cert = crypto.load_certificate(crypto.FILETYPE_PEM, f.read())

# Create a certificate store and add the root and intermediate CA certificates to it
store = crypto.X509Store()
store.add_cert(root_cert)
store.add_cert(int_cert)

# Create a certificate store context and set the store and client certificate to be verified
store_ctx = crypto.X509StoreContext(store, client_cert)

# Verify the client certificate
try:
    store_ctx.verify_certificate()
    print("Client certificate is valid.")
except Exception as e:
    print("Client certificate is invalid:", e)

Client certificate is invalid: [25, 2, 'path length constraint exceeded']


In [17]:
# Get the certificate chain
chain = store_ctx.get_chain()

# Count the number of intermediate CAs in the chain
num_intermediates = sum(1 for cert in cert_chain[1:-1] if 'CA' in cert.get_extension(0).get_data().decode())

# Check if the chain exceeds the path length constraint
if num_intermediates > 1:
    print("Certificate chain exceeds path length constraint")

AttributeError: 'X509StoreContext' object has no attribute 'get_chain'

In [15]:
# Load the root CA certificate
with open("root.crt", "rb") as f:
    root_cert = crypto.load_certificate(crypto.FILETYPE_PEM, f.read())

# Load the intermediate CA certificate
with open("int.crt", "rb") as f:
    int_cert = crypto.load_certificate(crypto.FILETYPE_PEM, f.read())

# Load the client certificate to be verified
with open("client.crt", "rb") as f:
    client_cert = crypto.load_certificate(crypto.FILETYPE_PEM, f.read())

# Create a certificate store and add the root and intermediate CA certificates to it
store = crypto.X509Store()
store.add_cert(root_cert)
store.add_cert(int_cert)

# Create a certificate store context and set the store and client certificate to be verified
store_ctx = crypto.X509StoreContext(store, client_cert)

# Verify the client certificate
try:
    store_ctx.verify_certificate()
    print("Client certificate is valid.")

    # Get the chain of certificates used to verify the client certificate
    chain = store_ctx.get0_chain()
    print("Chain of certificates used to verify the client certificate:")
    for cert in chain:
        print(cert.get_subject().CN)

except Exception as e:
    print("Client certificate is invalid:", e)

Client certificate is invalid: [25, 2, 'path length constraint exceeded']


In [8]:
# Load the root CA certificate
with open("root.crt", "rb") as f:
    root_cert = crypto.load_certificate(crypto.FILETYPE_PEM, f.read())

# Load the intermediate CA certificate
with open("int.crt", "rb") as f:
    int_cert = crypto.load_certificate(crypto.FILETYPE_PEM, f.read())

# Load the client certificate to be verified
with open("client.crt", "rb") as f:
    client_cert = crypto.load_certificate(crypto.FILETYPE_PEM, f.read())

# Create a certificate store and add the root and intermediate CA certificates to it
store = crypto.X509Store()
store.add_cert(root_cert)
store.add_cert(int_cert)

# Create a certificate context and set the store and client certificate to be verified
context = SSL.Context(SSL.TLSv1_2_METHOD)
context.load_verify_locations(cafile="root.crt")
context.load_cert_chain(certfile="client.crt")

# Verify the client certificate
store_ctx = crypto.X509StoreContext(store, client_cert)
store_ctx.verify_certificate()

# Print the result of the verification
print("Client certificate is valid.")

AttributeError: 'Context' object has no attribute 'load_cert_chain'

In [4]:
# Verify the client's certificate
client_cert = conn.get_peer_certificate()
if not client_cert:
    raise Exception("No certificate presented by the client")

# Verify the certificate's signature
try:
    crypto.X509StoreContext(store, client_cert).verify_certificate()
except crypto.X509StoreContextError as e:
    raise Exception("Invalid certificate: {}".format(e))

# Verify the certificate's common name
common_name = client_cert.get_subject().CN
if common_name != "www.example.com":
    raise Exception("Invalid common name: {}".format(common_name))

print("Successfully verified client's certificate!")

NameError: name 'conn' is not defined

In [5]:
store = crypto.X509Store()
store.add_cert(int_cert)
store.add_cert(root_cert)
ctx = SSL.Context(SSL.TLSv1_2_METHOD)
ctx.set_verify(SSL.VERIFY_PEER, lambda conn, cert, errno, depth, ok: ok)
ctx.load_verify_locations("int.crt", "root.crt")
ctx.set_cert_store(store)
conn = SSL.Connection(ctx)
conn.set_connect_state()
conn.set_tlsext_host_name("www.example.com".encode("utf-8"))
conn.set_cert_chain_file("client.crt")
conn.set_private_key_file("client.key")
conn.set_verify(SSL.VERIFY_PEER, lambda conn, cert, errno, depth, ok: ok)
conn.connect(("www.example.com", 443))
conn.do_handshake()
print("Successfully verified certificate!")

AttributeError: 'Context' object has no attribute 'set_cert_store'